1. Read observers raster data.

In [27]:
from osgeo import gdal
import numpy as np

data_path = '0_data/'

obs_blocks = gdal.Open(data_path + 'obs_blocks.tif')
obs_blocks_band = obs_blocks.GetRasterBand(1)
obs_blocks_data = obs_blocks.ReadAsArray()
row_length = obs_blocks.RasterYSize
col_length = obs_blocks.RasterXSize
tif_geotrans = obs_blocks.GetGeoTransform()
tif_proj = obs_blocks.GetProjection()

obs_values = []
obs_points = []
for row in range(row_length):
    for col in range(col_length):
        obs_value = obs_blocks_data[row, col]
        if obs_value > 0:
            obs_points.append([row, col])
            obs_values.append([int(obs_value)])
            
obs_values = np.array(obs_values)
obs_points = np.array(obs_points)

obs_max = obs_values.max()
obs_value_x = np.arange(obs_max + 1)
obs_value_numy = np.zeros(obs_max + 1)
for value in obs_values:
    obs_value_x[value] = value
    obs_value_numy[value] = obs_value_numy[value] + 1

2. Calculate outlier value.

In [28]:
Q1 = np.percentile(obs_values, 25)
Q3 = np.percentile(obs_values, 75)
outlier = Q3 + 1.5 * (Q3-Q1)

outlier_num = 0
for value in obs_values:
    if value > outlier:
        outlier_num = outlier_num + 1


# Only save outlier data
obs_outliers = np.zeros((row_length, col_length))
for row in range(row_length):
    for col in range(col_length):
        obs_value = obs_blocks_data[row, col]
        if obs_value > outlier:
            obs_outliers[row, col] = 1

# Remove noise of outlier data.            
obs_outliers_nonoise = np.zeros((row_length, col_length))      
for row in range(1, row_length-1):
    for col in range(1, col_length-1):
        obs_outif = obs_outliers[row, col]
        if obs_outif > 0:
            nbr_outsum = 0
            for k in [-1, 0, 1]:
                for l in [-1, 0, 1]:
                    obs_nbr_outif = obs_outliers[row+k, col+l]
                    nbr_outsum = nbr_outsum + obs_nbr_outif
            if nbr_outsum > 4:
                obs_outliers_nonoise[row, col] = 1

In [3]:
# Save to tif
driver = gdal.GetDriverByName('GTiff')
obs_outliers_dataset = driver.Create(data_path + 'obs_outliers.tif', col_length, row_length, 1, gdal.GDT_Int32)
obs_outliers_dataset.SetGeoTransform(tif_geotrans)
obs_outliers_dataset.SetProjection(tif_proj)
obs_outliers_dataset.GetRasterBand(1).WriteArray(obs_outliers)
# Save finish.
del obs_outliers_dataset

# Save to tif
driver = gdal.GetDriverByName('GTiff')
obs_outdenoise_dataset = driver.Create(data_path + 'obs_outliers_denoise.tif', col_length, row_length, 1, gdal.GDT_Int32)
obs_outdenoise_dataset.SetGeoTransform(tif_geotrans)
obs_outdenoise_dataset.SetProjection(tif_proj)
obs_outdenoise_dataset.GetRasterBand(1).WriteArray(obs_outliers_nonoise)
# Save finish.
del obs_outdenoise_dataset

3. Calculate autocorrelation range for following clustering process.

In [52]:
obs_comb = np.hstack((obs_points, obs_values))
obs_samples = np.copy(obs_comb)
np.random.shuffle(obs_samples)

import skgstat as skg
variogram = skg.Variogram(coordinates = obs_samples[:, :2], values = obs_samples[:, 2], 
                              n_lags = 50, maxlag = 50)
sp_threshold = variogram.parameters[0]


In [53]:
sp_threshold

4.42610086052637